In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sqlalchemy import create_engine,MetaData,Table,Insert
from sqlalchemy.types import UserDefinedType
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from sqlalchemy import create_engine, Column, Integer, String, Text, Date, ForeignKey
from dotenv import load_dotenv
from sqlalchemy.orm import sessionmaker
# Import the vector type if using pgvector
from pgvector.sqlalchemy import Vector
import os
import json
import requests

c:\Users\ajukh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
csvdata = pd.read_csv('./AdvizorPro_Person.csv',low_memory=False)
# Create a base class for declarative models
Base = declarative_base()
df = pd.DataFrame(csvdata)
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

c:\Users\ajukh\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
class BrokerData(Base):
    __tablename__ = 'brokercheck_data'
    id =  Column(String(50), primary_key=True)  # Primary key with text ID
    advisory_text = Column(Text)
    embedding = Column(Vector(384))

db_url = os.getenv("DATABASE_URI")
engine = create_engine(db_url)
Base.metadata.create_all(engine)
metadata = MetaData()

In [5]:
class Scrape_Data():
    def __init__(self,CRD):
        
        self.CRD = CRD
        # self.Call_API()
    def Call_API(self):
        print(f"The Calling the API for CRD =  {self.CRD} ")

        url = f'https://api.brokercheck.finra.org/search/individual/{self.CRD}'
        try:
            response = requests.get(url, timeout=10) 
            response.raise_for_status()  
        except requests.exceptions.RequestException as e:
           print(f"Error fetching data for CRD Number {self.CRD}: {e}")
           return False,False,False

        self.jsonresponse = response.json()
        if self.jsonresponse['hits']['total'] >0:
            return self.ExtractData()
        
    def ExtractData(self):
        
        contentdata = self.jsonresponse['hits']['hits'][0]['_source']['content']
        emb = model.encode(contentdata)
        return True,emb,contentdata

In [6]:
x = 0
for i in df.index:
    if x >= 1725:
        crd = df.loc[i,'CRD']
        data = Scrape_Data(crd) 
        res,emb,contentdata = data.Call_API()
        if res == True:
            Session = sessionmaker(bind=engine)
            session = Session()
            emb_list = emb.astype(float).tolist()
            insertdata = BrokerData(
                id =str(crd), # Primary key with text ID
                advisory_text = contentdata,
                embedding = emb_list
            )
            session.merge(insertdata)
            session.commit()
            # Close the session
            session.close()
            print(f"{crd} embedding inserted successfully")
    else:
        x += 1
    

The Calling the API for CRD =  1129157 
1129157 embedding inserted successfully
The Calling the API for CRD =  1129174 
1129174 embedding inserted successfully
The Calling the API for CRD =  1129265 
1129265 embedding inserted successfully
The Calling the API for CRD =  1129298 
1129298 embedding inserted successfully
The Calling the API for CRD =  1129330 
1129330 embedding inserted successfully
The Calling the API for CRD =  1129345 
1129345 embedding inserted successfully
The Calling the API for CRD =  1129349 
1129349 embedding inserted successfully
The Calling the API for CRD =  1129399 
1129399 embedding inserted successfully
The Calling the API for CRD =  1129451 
1129451 embedding inserted successfully
The Calling the API for CRD =  1129515 
1129515 embedding inserted successfully
The Calling the API for CRD =  1129757 
1129757 embedding inserted successfully
The Calling the API for CRD =  1129845 
1129845 embedding inserted successfully
The Calling the API for CRD =  1130019 
